Я вижу два решения:

Для простоты я бы сделал 2 класса датасета
Если нужно кодировать лишь последовательность букв, он и хранит последовательности букв сразу и не хранит коордианты

В обоих случаях декодер оперирует эмбеддингами букв текста

<h3>1. На вход энкодера x, y, t, dx/dt, dy/st, x'', y'', keybard_key_embedding</h3>

**Что делать, если ближайшая клавиша неалфавитная (пунктуация, клавиши-действия)?**

Добавлю для всех неалфавитных клавиш один специальный токен. Хотя, возможно,
лучше добавить отдельный токен для каждой клавиши. Кажется, в кавиатуре схожесть
клавиш определяется тем, насколько часто они встречаются рядом друг с другом.
Тогда может быть важно отличать enter, который близок к бкве `э`, например,
от `caps lock`, который близок к `ф`. Отмечу, что схожесть в данном случае - это
не просто физическое расстояние между клавишами (хотя отчасти и так), но скорее, похожесть
клавиш `a` и `b` означает, что для последовательностей вида `letter1`, `letter2` ... `letterX`, `letterN`
если вероятность, что `letterX` = `a` велика, то вероятность, что `letterX` = `b` тоже велика.


**Где происходит инициализация токенизатора?**
я бы вынес токенезатор вне датасета и передавал бы его в конструктор датасета.


для каждой раскладки свои instance'ы датасета и модели.



<h3> 2. На вход энкодера последовательность клавиш клавиатуры </h3>

Если ближайшая клавиша неалфавитная **пропускать**

**Где происходит инициализация токенизатора?**


один instance датасета и одна модель для всех раскладок.


Реализовывать ли для каждого варианта отдельный токенизатор:

У нас может быть различное количество токенов: в некотоорых раскладках отсутствует символ "ъ", например

Когда датасет содержит лишь одну раскладку, токенизатор должен учесть символы из одной раскладки. Когда датасет содержит несколько раскладок, токенизатор должен учесть символы из всех раскладок.

Кажется, что варьируется только наличие 'ъ' и 'ё'. Во-первых, не ясно нужны ли эти символы. Есть желание заменять 'ё' на 'е', а 'ъ' на 'ь'. 

Кажется, нас совершенно устраивает токенизатор, содержащий все буквы русского языка, включая 'ё' и 'ъ'. Наличие пары лишних токенов незначительно увеличит размер эмбеддинг-матрицы, но не повлияет на обучение модели.

In [25]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

In [6]:
# предполагается, что есть таблица predictors_table

# * predictor_id
# * word_generator_type
# * Generator_call_kwargs_json
# * Model_architecture_name
# * Model_weights_path
# * Grid_name
# * test_preds_path
# * val_preds_path
# * validation_metric

# predictor_id should autoincrement
df = pd.DataFrame(columns=[
    'predictor_id',
    'word_generator_type',
    'generator_call_kwargs_json',
    'model_architecture_name',
    'model_weights_path',
    'grid_name',
    'test_preds_path',
    'val_preds_path',
    'validation_mmr'])

df.to_csv('predictors_table.csv', index=False)

df = pd.read_csv('predictors_table.csv')

df


,predictor_id,word_generator_type,generator_call_kwargs_json,model_architecture_name,model_weights_path,grid_name,test_preds_path,val_preds_path,validation_mmr


In [7]:
print(df.keys())

Index(['predictor_id', 'word_generator_type', 'generator_call_kwargs_json',
       'model_architecture_name', 'model_weights_path', 'grid_name',
       'test_preds_path', 'val_preds_path', 'validation_mmr'],
      dtype='object')


In [8]:
dict = {
    'word_generator_type': ['word_generator_type1', 'word_generator_type2'],
    'generator_call_kwargs_json': ['generator_call_kwargs_json1', 'generator_call_kwargs_json2'],
    'model_architecture_name': ['model_architecture_name1', 'model_architecture_name2'],
    'model_weights_path': ['model_weights_path1', 'model_weights_path2'],
    'grid_name': ['grid_name1', 'grid_name2']
}

df1 = pd.DataFrame(dict)
display(df1)

,word_generator_type,generator_call_kwargs_json,model_architecture_name,model_weights_path,grid_name
0,word_generator_type1,generator_call_kwargs_json1,model_architecture_name1,model_weights_path1,grid_name1
1,word_generator_type2,generator_call_kwargs_json2,model_architecture_name2,model_weights_path2,grid_name2


In [9]:
pd.concat([df, df1], ignore_index = True)

,predictor_id,word_generator_type,generator_call_kwargs_json,model_architecture_name,model_weights_path,grid_name,test_preds_path,val_preds_path,validation_mmr
0,NaN,word_generator_type1,generator_call_kwargs_json1,model_architecture_name1,model_weights_path1,grid_name1,NaN,NaN,NaN
1,NaN,word_generator_type2,generator_call_kwargs_json2,model_architecture_name2,model_weights_path2,grid_name2,NaN,NaN,NaN


In [28]:
import os
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, IterableDataset
from tqdm import tqdm

from model import SwipeCurveTransformer, get_m1_model
from tokenizers import CharLevelTokenizerv2, KeyboardTokenizerv1
from dataset import NeuroSwipeDatasetv3
from word_generators import GreedyGenerator, BeamGenerator
from metrics import get_mmr

In [29]:
IN_KAGGLE = False

if IN_KAGGLE:
    DATA_ROOT = "/kaggle/input/yandex-cup-playground"
    MODELS_DIR = ""
else:
    DATA_ROOT = "../data/data_separated_grid"
    MODELS_DIR = "../data/trained_models/m1"

In [30]:
import pickle

word_tokenizer = CharLevelTokenizerv2(os.path.join(DATA_ROOT, "voc.txt"))

if not IN_KAGGLE:
    word_tokenizer_save_path = os.path.join(DATA_ROOT, "word_tokenizer.pkl")

    with open(word_tokenizer_save_path, 'wb') as f:
        pickle.dump(word_tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)

    with open(word_tokenizer_save_path, 'rb') as f:
        word_tokenizer = pickle.load(f)

In [31]:
print(word_tokenizer.idx_to_char)

{0: '-', 1: 'а', 2: 'б', 3: 'в', 4: 'г', 5: 'д', 6: 'е', 7: 'ж', 8: 'з', 9: 'и', 10: 'й', 11: 'к', 12: 'л', 13: 'м', 14: 'н', 15: 'о', 16: 'п', 17: 'р', 18: 'с', 19: 'т', 20: 'у', 21: 'ф', 22: 'х', 23: 'ц', 24: 'ч', 25: 'ш', 26: 'щ', 27: 'ъ', 28: 'ы', 29: 'ь', 30: 'э', 31: 'ю', 32: 'я', 33: '<eos>', 34: '<unk>', 35: '<pad>', 36: '<sos>'}


In [32]:
def get_grid(grid_name: str, grids_path: str) -> dict:
    with open(grids_path, "r", encoding="utf-8") as f:
        return json.load(f)[grid_name]

In [36]:
grid_name = "extra"
sample_data = os.path.join(DATA_ROOT, f"valid__in_train_format__{grid_name}_only.jsonl")
grid_path =  os.path.join(DATA_ROOT, "gridname_to_grid.json")

grid = get_grid(grid_name, grid_path)
kb_tokenizer = KeyboardTokenizerv1()
word_tokenizer = CharLevelTokenizerv2(os.path.join(DATA_ROOT, "voc.txt"))


dataset = NeuroSwipeDatasetv3(
    data_path = sample_data,
    gridname_to_grid= {grid_name: grid},
    kb_tokenizer = kb_tokenizer,
    word_tokenizer = word_tokenizer,
    include_time = False,
    include_velocities = True,
    include_accelerations = True,
    total = 9_416
)

In [ ]:
dataset[0]

In [12]:
# full train dataset before adding nearest_kb_label:
# ----------------------------------
# 16.2GB RAM (when data stored as torch tensors)
# when stored as lists 17Gb is 61%
# when data stored as python arrays with dtype short: 4.8GB RAM


# Now:
# ----
# 9.3 Gb RAM

In [468]:
i = 40
(xyt, kb_tokens, dec_in_char_seq, traj_pad_mask, word_pad_mask), dec_out_char_seq = dataset[i]
print(xyt.shape, kb_tokens.shape, dec_in_char_seq.shape, traj_pad_mask.shape, word_pad_mask.shape, dec_out_char_seq.shape)

torch.Size([299, 6]) torch.Size([299]) torch.Size([35]) torch.Size([299]) torch.Size([35]) torch.Size([35])


In [469]:
with np.printoptions(threshold=1000000):
    print(dataset._nearest_kb_label_dict['default'][:3])

[['й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й' 'й'
  'й' 'й' 'й' 'й' 'й' 'й' 'й' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф'
  'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф'
  'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф'
  'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф'
  'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 'ф' 

In [470]:
# def get_mask(max_seq_len: int):
#     """
#     Returns a mask for the decoder transformer.
#     """
#     mask = torch.triu(torch.ones(max_seq_len, max_seq_len), diagonal=1)
#     mask = mask.masked_fill(mask == 1, float('-inf'))
#     return mask

# mask = get_mask(5)
# print(mask)

# >>>
# tensor([[0., -inf, -inf, -inf, -inf],
#         [0., 0., -inf, -inf, -inf],
#         [0., 0., 0., -inf, -inf],
#         [0., 0., 0., 0., -inf],
#         [0., 0., 0., 0., 0.]])

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
!python model_tests.py

Encoder out shape correct
Decoder out shape correct


In [11]:
word_char_tokenizer = CharLevelTokenizerv1(os.path.join(DATA_ROOT, "voc.txt"))

In [475]:
word_char_tokenizer.tokenize("троллейбус")

(tensor([36, 19, 17, 15, 12, 12,  6, 10,  2, 20, 18, 33, 34, 34, 34, 34, 34, 34,
         34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34]),
 tensor([False, False, False, False, False, False, False, False, False, False,
         False, False,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True]))

In [476]:
print(word_char_tokenizer.char_to_idx.keys())

dict_keys(['-', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', '<eos>', '<pad>', '<unk>', '<sos>'])


In [477]:
len(word_char_tokenizer.tokenize('информационно-телекоммуникационной')[0])

36

In [478]:
len(word_char_tokenizer.tokenize('информационно')[0])

36

In [479]:
word_char_tokenizer.decode(word_char_tokenizer.tokenize('информационно-телекоммуникационной')[0])

'<sos>информационно-телекоммуникационной<eos>'

In [480]:
word_char_tokenizer.max_word_len - 1

35

In [12]:
def prepare_batch_v1(x, y, device):
    (xyt, kb_tokens, dec_in_char_seq, traj_pad_mask, word_pad_mask), dec_out_char_seq = x, y
    
    if xyt is not None:
        xyt = xyt.transpose_(0, 1).to(device)  # (curves_seq_len, batch_size, n_coord_feats)
    if kb_tokens is not None:
        kb_tokens = kb_tokens.transpose_(0, 1).to(device) # (curves_seq_len, batch_size)
    if dec_in_char_seq is not None:
        dec_in_char_seq = dec_in_char_seq.transpose_(0, 1).to(device)  # (chars_seq_len - 1, batch_size)
    if dec_out_char_seq is not None:
        dec_out_char_seq = dec_out_char_seq.transpose_(0, 1).to(device)  # (chars_seq_len - 1, batch_size)

    if traj_pad_mask is not None:
        traj_pad_mask = traj_pad_mask.to(device)  # (batch_size, curves_seq_len)
    if word_pad_mask is not None:
        word_pad_mask = word_pad_mask.to(device)  # (batch_size, chars_seq_len - 1)

    return (xyt, kb_tokens, dec_in_char_seq, traj_pad_mask, word_pad_mask), dec_out_char_seq

prepare_batch = prepare_batch_v1

In [13]:
def turncate_traj_batch(xyt, kb_tokens, traj_pad_mask):
    max_curve_len = int(torch.max(torch.sum(~traj_pad_mask, dim = 1)))
    xyt = xyt[:, :max_curve_len]
    kb_tokens = kb_tokens[:, :max_curve_len]
    traj_pad_mask = traj_pad_mask[:, :max_curve_len]
    return xyt, kb_tokens, traj_pad_mask



def prepare_batch_with_pad_truncation(x, y, device):
    (xyt, kb_tokens, dec_in_char_seq, traj_pad_mask, word_pad_mask), dec_out_char_seq = x, y

    xyt, kb_tokens, traj_pad_mask = turncate_traj_batch(xyt, kb_tokens, traj_pad_mask)

    # print(max_curve_len)

    xyt = xyt.transpose_(0, 1).to(device)  # (curves_seq_len, batch_size, n_coord_feats)
    kb_tokens = kb_tokens.transpose_(0, 1).to(device) # (curves_seq_len, batch_size)
    dec_in_char_seq = dec_in_char_seq.transpose_(0, 1).to(device)  # (chars_seq_len - 1, batch_size)
    dec_out_char_seq = dec_out_char_seq.transpose_(0, 1).to(device)  # (chars_seq_len - 1, batch_size)

    traj_pad_mask = traj_pad_mask.to(device)  # (batch_size, max_curve_len)
    # traj_pad_mask = torch.zeros_like(kb_tokens, dtype = torch.bool).transpose_(0, 1).to(device)
    word_pad_mask = word_pad_mask.to(device)  # (batch_size, chars_seq_len - 1)

    return (xyt, kb_tokens, dec_in_char_seq, traj_pad_mask, word_pad_mask), dec_out_char_seq

# prepare_batch = prepare_batch_with_pad_truncation

In [481]:
max_out_seq_len = word_char_tokenizer.max_word_len - 1

transformer = SwipeCurveTransformer(
    n_coord_feats=6,
    char_emb_size=128,
    char_vocab_size=len(word_char_tokenizer.char_to_idx),
    key_emb_size=54,
    num_encoder_layers=4,
    num_decoder_layers=3,
    dim_feedforward=128,
    num_heads_encoder_1=4,
    num_heads_encoder_2=4,
    num_heads_decoder=4,
    dropout=0.1,
    char_embedding_dropout=0.1,
    key_embedding_dropout=0.1,
    max_out_seq_len=max_out_seq_len,
    max_curves_seq_len=299,
    device = device)

In [482]:
import torch.nn.functional as F

def cross_entropy_with_reshape(pred, target):
    """
    pred - BatchSize x TargetLen x VocabSize
    target - BatchSize x TargetLen
    """
    pred_flat = pred.view(-1, pred.shape[-1])  # BatchSize*TargetLen x VocabSize
    target_flat = target.reshape(-1)  # BatchSize*TargetLen
    return F.cross_entropy(pred_flat, target_flat, ignore_index=0)

In [483]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=10, shuffle=True)


for x, y in loader:
    x, y = prepare_batch(x, y, device)

    char_seq_pred = transformer(*x)

    cross_entropy_with_reshape(char_seq_pred, y)

    
    break

c:\Users\proshian\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


In [484]:
char_seq_pred.transpose(0,1)[0].shape

if True in torch.isnan(char_seq_pred):
    print(char_seq_pred)

In [485]:
print(char_seq_pred.shape)  # max_word_len - 1, batch_size, char_vocab_size - 2 (omitting <sos> and <pad>)

torch.Size([35, 10, 35])


In [486]:
def lr_scheduler(optimizer):
    return torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                      patience=20,
                                                      factor=0.5,
                                                      verbose=True)

In [487]:
def init_random_seed(value=42):
    # random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)
    torch.cuda.manual_seed(value)
    # torch.backends.cudnn.deterministic = True

In [14]:
model = get_m1_model(
    os.path.join(MODELS_DIR,
                 'best_model__2023_11_04__18_31_37__0.02530_default_switch_2.pt'),
    device
)

In [493]:
grid_name_to_grid_path = os.path.join(DATA_ROOT, "gridname_to_grid.json")
with open(grid_name_to_grid_path, "r", encoding="utf-8") as f:
    grid_name_to_grid = json.load(f)

test_path = os.path.join(DATA_ROOT, f"test.jsonl")

kb_tokenizer = KeyboardTokenizerv1()
word_tokenizer = CharLevelTokenizerv1(os.path.join(DATA_ROOT, "voc.txt"))


test_dataset = NeuroSwipeDatasetv1(
    data_path = test_path,
    gridname_to_grid = grid_name_to_grid,
    kb_tokenizer = kb_tokenizer,
    max_traj_len = 299,
    word_tokenizer = word_tokenizer,
    include_time = False,
    include_velocities = True,
    include_accelerations = True,
    has_target=False,
    has_one_grid_only=False,
    include_grid_name=True,
    total = 10_000
)

100%|██████████| 10000/10000 [00:04<00:00, 2196.84it/s]


In [55]:
grid_name_to_grid_path = os.path.join(DATA_ROOT, "gridname_to_grid.json")
with open(grid_name_to_grid_path, "r", encoding="utf-8") as f:
    grid_name_to_grid = json.load(f)

val_path = os.path.join(DATA_ROOT, f"valid__in_train_format.jsonl")

kb_tokenizer = KeyboardTokenizerv1()
word_tokenizer = CharLevelTokenizerv1(os.path.join(DATA_ROOT, "voc.txt"))


val_dataset = NeuroSwipeDatasetv1(
    data_path = val_path,
    gridname_to_grid = grid_name_to_grid,
    kb_tokenizer = kb_tokenizer,
    max_traj_len = 299,
    word_tokenizer = word_tokenizer,
    include_time = False,
    include_velocities = True,
    include_accelerations = True,
    has_target=False,
    has_one_grid_only=False,
    include_grid_name=True,
    total = 10_000
)

100%|██████████| 10000/10000 [00:03<00:00, 2994.91it/s]


In [16]:
default_model_fname = "best_model__2023_11_03__20_42_24__0.02608_default_switch_1.pt"
extra_model_fname = "best_model__2023_11_05__07_55_13__0.02516_extra_switch_2__with_pad_cutting.pt"

grid_name_to_model = {
    "default": get_m1_model(os.path.join(MODELS_DIR, default_model_fname), device),
    "extra": get_m1_model(os.path.join(MODELS_DIR, extra_model_fname), device)
}

In [501]:
grid_name_to_greedy_generator_v1 = {
    grid_name: GreedyGenerator(grid_name_to_model[grid_name], word_char_tokenizer, device)
    for grid_name in ("default", "extra")
}

In [50]:
def create_submission_greedy(dataset,
                             grid_name_to_greedy_generator,
                             baseline_preds_path, 
                             out_path,
                             vocab_set):
    """
    Creates submission file generating words greedily.

    If prediction is not in the vocabulary 
    """

    if os.path.exists(out_path):
        raise ValueError(f"File {out_path} already exists")
    
    with open(out_path, "w", encoding="utf-8") as f:
        with open(baseline_preds_path, "r", encoding="utf-8") as baseline_f:
            for i, (data, baseline_str) in tqdm(enumerate(zip(dataset, baseline_f)), total=len(dataset)):
                (xyt, kb_tokens, _, traj_pad_mask, _), _, grid_name = data
                pred = grid_name_to_greedy_generator[grid_name](xyt, kb_tokens, traj_pad_mask)
                pred = pred.removeprefix("<sos>") 
                if pred in vocab_set:
                    pred_list = [pred, *baseline_str.split(",")[:-1]]
                    pred_str = ",".join(pred_list)
                    f.write(pred_str + "\n")
                else:
                    f.write(baseline_str)

In [65]:
def create_pred_list_beam(dataset,
                          grid_name_to_beam_generator,
                          beamsize,
                          verbose):
    """
    Creates submission file generating words greedily.

    If prediction is not in the vocabulary 
    """
    all_pred_list = [None]*len(dataset)

    for i, data in tqdm(enumerate(dataset), total=len(dataset)):
        try:
            (xyt, kb_tokens, _, traj_pad_mask, _), _, grid_name = data
            pred_list = grid_name_to_beam_generator[grid_name](
                xyt, kb_tokens, traj_pad_mask, max_steps_n = 35, beamsize=beamsize, verbose=verbose)
            pred_list = [pred for score, pred in pred_list]
            all_pred_list[i] = pred_list

            if not pred_list:
                print(f"grid_name = {grid_name}")
            
        except KeyboardInterrupt:
            print('Досрочно остановлено пользователем')
            break
    return all_pred_list

In [26]:
def get_vocab_set(vocab_path: str):
    with open(vocab_path, 'r', encoding = "utf-8") as f:
        return set(f.read().splitlines())

In [27]:
vocab_set = get_vocab_set(os.path.join(DATA_ROOT, "voc.txt"))

In [55]:
prepare_batch = prepare_batch_v1

In [ ]:
# clean_result = []
# for score, word in result:
#     word = word.removeprefix("<sos>").removesuffix("<eos>")
#     if word not in self.voc_set:
#         continue
#     clean_result.append((score, word))
#     if len(clean_result) == return_hypotheses_n:
#         break

# if not clean_result:
#     print("Clean result is empty! Original result:")
#     print(result) 
# # result = result[:return_hypotheses_n]

# if verbose:
#     print(clean_result)


# if not pred_list:
#     print(f"grid_name = {grid_name}")

In [211]:
def create_raw_pred_list_beam(dataset,
                          grid_name_to_beam_generator,
                          beamsize,
                          verbose):
    """
    Creates submission file generating words greedily.

    If prediction is not in the vocabulary 
    """
    all_pred_list = [None]*len(dataset)
    
    RETURN_HYPOTHESIS_N = 8

    for i, data in tqdm(enumerate(dataset), total=len(dataset)):
        try:
            (xyt, kb_tokens, _, traj_pad_mask, _), _, grid_name = data
            pred_list = grid_name_to_beam_generator[grid_name](
                xyt, kb_tokens, traj_pad_mask, max_steps_n = 35,
                beamsize=beamsize, verbose=verbose, return_hypotheses_n = RETURN_HYPOTHESIS_N)
            all_pred_list[i] = pred_list

            
        except KeyboardInterrupt:
            print('Досрочно остановлено пользователем')
            break
    return all_pred_list

In [218]:
default_model_fname = "best_model__2023_11_04__18_31_37__0.02530_default_switch_2.pt"
extra_model_fname = "best_model__2023_11_05__07_55_13__0.02516_extra_switch_2__with_pad_cutting.pt"

grid_name_to_model = {
    "default": get_m1_model(os.path.join(MODELS_DIR, default_model_fname), device),
    "extra": get_m1_model(os.path.join(MODELS_DIR, extra_model_fname), device)
}

grid_name_to_beam_generator = {
    grid_name: BeamGenerator(grid_name_to_model[grid_name], word_char_tokenizer, device)
    for grid_name in ("default", "extra")
}

raw_test_preds = create_raw_pred_list_beam(
    test_dataset,
    grid_name_to_beam_generator,
    beamsize = 5,
    verbose = False
)

  0%|          | 0/10000 [00:00<?, ?it/s]C:\Users\SystemPoint\AppData\Local\Temp\ipykernel_16712\1154242103.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  next_tokens_logproba = F.log_softmax(next_tokens_logits)
100%|██████████| 10000/10000 [2:20:58<00:00,  1.18it/s] 


In [220]:
test_raw_preds_path = os.path.join(DATA_ROOT, f"test_raw_pred___{default_model_fname}__{extra_model_fname}.pkl")

with open(test_raw_preds_path, 'wb') as f:
    pickle.dump(raw_test_preds, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
grid_name_to_beam_generator = {
    grid_name: BeamGenerator(grid_name_to_model[grid_name], word_char_tokenizer, vocab_set, device)
    for grid_name in ("default", "extra")
}

test_preds = create_pred_list_beam(
    test_dataset,
    grid_name_to_beam_generator,
    beamsize = 5,
    verbose = False
)

In [67]:
test_preds[:69]

[['на', 'неа', 'не', 'нас'],
 ['что', 'сто', 'чтоб', 'сито'],
 ['опоздания', 'опоздание', 'опоздании', 'опозданий'],
 ['сколько', 'скольки', 'скольких', 'сколь'],
 ['дремать', 'донимать', 'делать'],
 ['не', 'нее', 'ну', 'на'],
 ['как', 'кака', 'какие', 'како'],
 ['садовод', 'садовода', 'садов', 'салов'],
 ['заметил', 'заметили', 'заметила', 'замерил'],
 ['ваши', 'ваги', 'аги', 'вани'],
 ['ок', 'он', 'оке', 'ока'],
 [],
 ['ай', 'ау', 'вай', 'айк'],
 ['ищем', 'ищет', 'идем', 'измен'],
 ['они', 'лет', 'он', 'оно'],
 ['могу', 'могут', 'могул', 'мог'],
 ['может', 'можете', 'молет', 'можно'],
 ['спокойный', 'спокойной', 'спокойные', 'спокойней'],
 ['рядом', 'ряды', 'рядов', 'рядовые'],
 ['вспомнить', 'вспомнилась', 'вспомнил', 'вспоминать'],
 ['максим', 'макам', 'маками', 'максам'],
 ['веселое', 'веселом', 'веселов', 'веселовой'],
 [],
 ['туда', 'труда', 'тула'],
 ['тебя', 'тебе'],
 ['ре', 'он', 'рен', 'не'],
 ['точно', 'точное', 'точном', 'точной'],
 ['чего', 'чел', 'чело', 'чанг'],
 ['помо

In [ ]:
test_preds[:13]

[['на', 'неа', 'не', 'нас'],
 ['что', 'сто', 'чтоб', 'сито'],
 ['опоздания', 'опоздание', 'опоздании', 'опоздает'],
 ['сколько', 'скольки', 'скольких', 'сколь'],
 ['дремать', 'донимать'],
 ['не', 'нее', 'ну', 'на'],
 ['как', 'кака', 'какие', 'како'],
 ['садовод', 'садов'],
 ['заметил', 'заметили', 'заметила', 'заметит'],
 ['ваши', 'ваги', 'аги', 'вани'],
 ['ок', 'он', 'оке', 'ока'],
 [],
 ['ай', 'ау', 'айк', 'айкай']]

In [ ]:
grid_name_to_beam_generator = {
    grid_name: BeamGenerator(grid_name_to_model[grid_name], word_char_tokenizer, vocab_set, device)
    for grid_name in ("default", "extra")
}

val_preds = create_pred_list_beam(
    val_dataset,
    grid_name_to_beam_generator,
    beamsize = 4,
    verbose = False
)

In [75]:
val_preds[:34]

[['на', 'нас', 'неа', 'нам'],
 ['все', 'всем', 'вся', 'всей'],
 ['этом', 'этому', 'этот', 'этим'],
 ['добрый', 'добрые', 'добрым'],
 ['девочка', 'девочки'],
 ['сказала', 'сказал', 'сказали', 'сказать'],
 ['скинь', 'скин'],
 ['геев'],
 ['тобой'],
 ['быстра', 'быстр', 'быстро', 'баса'],
 ['есть', 'если', 'ест', 'ес'],
 ['да', 'дора', 'дар', 'дал'],
 ['муж'],
 ['щас', 'щам', 'ща', 'щами'],
 ['она', 'они', 'оно', 'онам'],
 ['проблема', 'проблемы', 'проблемам', 'проблем'],
 ['билайн'],
 ['уже', 'уде', 'ужен', 'ужа'],
 ['раньше'],
 ['рам', 'нам', 'рас', 'пас'],
 ['щас', 'щам', 'ща'],
 ['купил', 'купила', 'купили', 'купило'],
 ['ты', 'бы', 'тв', 'был'],
 ['зовут'],
 ['короче', 'корочек', 'корочке', 'корочка'],
 ['лучше', 'лучшее', 'лучшей', 'лучшие'],
 ['приедем', 'придем', 'приедет', 'приеду'],
 ['размыто', 'размыт', 'рахмат'],
 ['давай', 'давайте'],
 ['ты', 'тв', 'та', 'тыс'],
 ['отдать', 'отжать', 'отдавать', 'отдаст'],
 ['привет', 'привете', 'привета', 'приветик'],
 ['не', 'нее', 'на', 'н

In [99]:
from datetime import datetime
cur_time = "{:%Y_%m_%d__%H_%M_%S}".format(datetime.now())

val_preds_save_path = os.path.join(DATA_ROOT, f"val_pred_{cur_time}.pkl")
test_preds_save_path = os.path.join(DATA_ROOT, f"test_pred_{cur_time}.pkl")

val_pred_dict = {
    "default": default_model_fname,
    "extra": extra_model_fname,
    "preds": val_preds
}

test_pred_dict = {
    "default": default_model_fname,
    "extra": extra_model_fname,
    "preds": test_preds
}

with open(val_preds_save_path, 'wb') as f:
    pickle.dump(val_pred_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(test_preds_save_path, 'wb') as f:
    pickle.dump(test_pred_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

In [111]:
with open(os.path.join(DATA_ROOT, "valid.ref"), 'r', encoding = "utf-8") as f:
    val_reference = f.read().splitlines()
val_reference[:34]

['на',
 'все',
 'этом',
 'добрый',
 'девочка',
 'сказала',
 'скинь',
 'геев',
 'тобой',
 'была',
 'есть',
 'да',
 'муж',
 'щас',
 'она',
 'проблема',
 'билайн',
 'уже',
 'раньше',
 'рам',
 'щас',
 'купил',
 'ты',
 'зовут',
 'короче',
 'лучше',
 'приедем',
 'размыто',
 'давай',
 'ты',
 'отдать',
 'привет',
 'не',
 'да']

при beamsize = 4 на Kaggle:
примерно 4 часа без P100 и 3 часа с P100
Если использовать prepare_batch_with_pad_clipping часа

In [74]:
from tqdm import tqdm

In [59]:
grid_name_to_greedy_generator = {
    grid_name: GreedyGenerator(grid_name_to_model[grid_name], word_char_tokenizer, device)
    for grid_name in ("default", "extra")
}

In [60]:
create_submission_greedy(
    test_dataset,
    grid_name_to_greedy_generator,
    baseline_preds_path = "../data/submissions/sample_submission.csv",
    out_path = "greedy_submionvdrp.csv",
    vocab_set = vocab_set
)

  1%|          | 53/10000 [00:03<09:57, 16.66it/s]


KeyboardInterrupt: 

In [84]:
with open(os.path.join(DATA_ROOT, "valid.ref"), 'r', encoding="utf-8") as f:
    ref_list = f.read().splitlines()
get_mmr(val_preds, ref_list)

0.8758890000000076

In [77]:
def augment_submission(preds, augment_list):
    for pred, aug in zip(preds, augment_list):
        for aug_el in aug:
            if len(pred) >= 4:
                break
            pred.append(aug_el)
    

In [86]:
augment_list = None
with open(r"..\data\submissions\sample_submission.csv", 'r', encoding = 'utf-8') as f:
    augment_lines = f.read().splitlines()
augment_list = [line.split(",") for line in augment_lines]

In [48]:
augment_submission(test_preds, augment_list)

NameError: name 'test_preds' is not defined

In [89]:
test_preds

[['на', 'неа', 'не', 'нас'],
 ['что', 'сто', 'чтоб', 'со'],
 ['опоздания', 'опоздание', 'опозданий', 'опоздании'],
 ['сколько', 'скольки', 'скольких', 'сколько'],
 ['донимать', 'деревья', 'дремать', 'дописать'],
 ['не', 'нее', 'ну', 'на'],
 ['как', 'кака', 'кан', 'какие'],
 ['садовод', 'садовода', 'садов', 'спародировал'],
 ['заметил', 'замерил', 'заметили', 'заметила'],
 ['ваги', 'аги', 'ваши', 'ани'],
 ['ок', 'он', 'оке', 'ока'],
 ['плинтус', 'плинтус', 'потренируемся', 'поинтересуемся'],
 ['ай', 'ау', 'айк', 'ава'],
 ['ищем', 'идем', 'ищет', 'ишемии'],
 ['они', 'оно', 'он', 'от'],
 ['могу', 'могут', 'могул', 'мог'],
 ['может', 'можете', 'молет', 'можем'],
 ['спокойный', 'спокойные', 'спокойной', 'спокойней'],
 ['рядом', 'ряды', 'рядов', 'рядовые'],
 ['вспомнить', 'вспомнил', 'вспоминать', 'вспомнилась'],
 ['максим', 'маками', 'макам', 'макс'],
 ['веселое', 'веселом', 'веселов', 'веселовой'],
 ['невинномысск', 'невинномысске', 'невинномысска', 'невинномысская'],
 ['туда', 'труда', 'т

In [90]:
for line in test_preds:
    if len(line) != 4:
        print(len(line))

In [76]:
def create_submission(preds_list, out_path):
    if os.path.exists(out_path):
        raise ValueError(f"File {out_path} already exists")
    
    with open(out_path, "w", encoding="utf-8") as f:
        for preds in preds_list:
            pred_str = ",".join(preds)
            f.write(pred_str + "\n")

In [95]:
create_submission(test_preds, f"submit_{cur_time}.csv")

In [186]:
import editdistance

In [187]:
closest_word = min(vocab_set, key=lambda word: editdistance.eval('бэтимэн', word))

In [188]:
min_dist = float('inf')
clsest_word = None
for vocab_word in vocab_set:
    dist = editdistance.eval('бэтимэн', vocab_word)
    if dist < min_dist:
        min_dist = dist
        closest_word = vocab_word

In [189]:
closest_word

'бэтмэн'

In [511]:
3 * 2000 / 60 * 5

100.0

In [181]:
with open(os.path.join(DATA_ROOT, 'val_pred_2023_11_07__02_25_25.pkl'), 'rb') as f:
    val_pred_dict = pickle.load(f)

In [183]:
val_pred_dict['default'], val_pred_dict['extra']

('best_model__2023_11_03__20_42_24__0.02608_default_switch_1.pt',
 'best_model__2023_11_05__07_55_13__0.02516_extra_switch_2__with_pad_cutting.pt')

In [184]:
# with open(os.path.join(DATA_ROOT, "valid.ref"), 'r', encoding="utf-8") as f:
#     ref_list = f.read().splitlines()
get_mmr(val_pred_dict['preds'], ref_list)

0.8758890000000076

# model evaluation using greedy search

In [107]:
grid_name_to_grid_path = os.path.join(DATA_ROOT, "gridname_to_grid.json")
with open(grid_name_to_grid_path, "r", encoding="utf-8") as f:
    grid_name_to_grid = json.load(f)

val_path = os.path.join(DATA_ROOT, f"valid__in_train_format.jsonl")

kb_tokenizer = KeyboardTokenizerv1()
word_tokenizer = CharLevelTokenizerv1(os.path.join(DATA_ROOT, "voc.txt"))


val_dataset = NeuroSwipeDatasetv1(
    data_path = val_path,
    gridname_to_grid = grid_name_to_grid,
    kb_tokenizer = kb_tokenizer,
    max_traj_len = 299,
    word_tokenizer = word_tokenizer,
    include_time = False,
    include_velocities = True,
    include_accelerations = True,
    has_target=True,
    has_one_grid_only=False,
    include_grid_name=True,
    total = 10_000
)

100%|██████████| 10000/10000 [00:04<00:00, 2498.47it/s]


In [143]:
def create_pred_list_greedy_witout_vocab_check(dataset,
                                               grid_name_to_greedy_generator,
                                               skip_grid_name = None):
    """
    Creates submission file generating words greedily.

    If prediction is not in the vocabulary 
    """

    preds = []
    targets = []

    for i, data in tqdm(enumerate(dataset), total=len(dataset)):
        try:
            (xyt, kb_tokens, _, traj_pad_mask, word_mask), target, grid_name = data
            if grid_name == skip_grid_name:
                continue
            pred = grid_name_to_greedy_generator[grid_name](xyt, kb_tokens, traj_pad_mask)
            pred = pred.removeprefix("<sos>") 
            target_len = torch.sum(~word_mask)
            target = word_char_tokenizer.decode(target[:target_len]).removesuffix('<pad>').removesuffix('<eos>')
            preds.append([pred])
            targets.append(target)
        except KeyboardInterrupt:
            print('Досрочно остановлено пользователем')
            break
    return preds, targets

In [153]:
def evaluate_weights(val_dataset,
                     model_getter,
                     grid_name,
                     weights_path,
                     device):
    assert grid_name in ("extra", "default")
    grid_name_to_greedy_generator = {grid_name:  GreedyGenerator(model_getter(weights_path, device), word_char_tokenizer, device)}
    skip_grid_name = "default" if grid_name == "extra" else "extra"
    preds, targets = create_pred_list_greedy_witout_vocab_check(val_dataset, grid_name_to_greedy_generator, skip_grid_name=skip_grid_name)
    mrr = get_mmr(preds, targets)
    return mrr

In [160]:
WEIGHTS_TO_FNAME_PATH = os.path.join(
    DATA_ROOT,
    "weights_fname_to_mrr.pkl"
)

In [205]:
weights_fname_to_mrr = {}
if os.path.exists(WEIGHTS_TO_FNAME_PATH):
    with open(WEIGHTS_TO_FNAME_PATH, 'rb') as f:
        weights_fname_to_mrr = pickle.load(f)

In [185]:
weights_fname_to_mrr

{'best_model__2023_11_03__20_42_24__0.02608_default_switch_1.pt': 0.8343245539507221,
 'best_model__2023_11_04__18_31_37__0.02530_default_switch_2.pt': 0.8369796091758709,
 'best_model__2023_11_03__16_34_37__0.02697_default_switch_1.pt': 0.828483432455395,
 'best_model__2023_11_05__07_55_13__0.02516_extra_switch_2__with_pad_cutting.pt': 0.8476027397260274}

In [206]:
from word_generators import GreedyGenerator

weights_fname = 'best_model__2023_11_03__16_34_37__0.02697_default_switch_1.pt'
weights_path = os.path.join(MODELS_DIR, weights_fname)
grid_name = "default"


if weights_fname not in weights_fname_to_mrr:
    mrr = evaluate_weights(val_dataset,
                            get_m1_model,
                            grid_name = grid_name,
                            weights_path = weights_path,
                            device = device)

    weights_fname_to_mrr[weights_fname] = mrr

print(weights_fname_to_mrr)

{'best_model__2023_11_03__20_42_24__0.02608_default_switch_1.pt': 0.8343245539507221, 'best_model__2023_11_04__18_31_37__0.02530_default_switch_2.pt': 0.8369796091758709, 'best_model__2023_11_03__16_34_37__0.02697_default_switch_1.pt': 0.828483432455395, 'best_model__2023_11_05__07_55_13__0.02516_extra_switch_2__with_pad_cutting.pt': 0.8476027397260274}


In [180]:
with open(WEIGHTS_TO_FNAME_PATH, 'wb') as f:
    pickle.dump(weights_fname_to_mrr, f, protocol=pickle.HIGHEST_PROTOCOL)

# Min dist algo

In [140]:
from typing import List, Tuple, Set, Dict
import heapq

from tqdm import tqdm
import editdistance


def get_real_and_errorous_words(preds: List[List[Tuple[float, str]]],
                                vocab_set: Set[str]) -> Tuple[List[List[str]], Dict[int, List[str]]]:
    """
    Arguments:
    ----------
    preds: List[List[Tuple[float, str]]]
        preds[i] stores raw output of a wordgenerator corresponding
        to the i-th curve. The raw output is a list of tuples, where
        each tuple is (-log_probability, word). The list is sorted
        by -log_probability in ascending order.
    vocab_set: Set[str]
        A set of all possible words.

    Returns:
    --------
    all_real_word_preds: List[List[str]]
        all_real_word_preds stores 4 lists of real words sorted by
        -log_probability in ascending order.
    all_errorous_word_preds: Dict[int, List[str]]
        all_errorous_word_preds[i] stores a list of errorous words
        sorted by -log_probability in ascending order if all_real_word_preds[i]
        has less than 4 words. Otherwise, all_errorous_word_preds does not
        have the key i.
    """

    all_real_word_preds = []
    all_errorous_word_preds = {}

    for i, pred in enumerate(preds):
        real_word_preds = []
        errorous_word_preds = []
        for _, word in pred:
            if word in vocab_set:
                real_word_preds.append(word)
                if len(real_word_preds) == 4:
                    break
            else:
                errorous_word_preds.append(word)
        
        all_real_word_preds.append(real_word_preds)
        if len(real_word_preds) < 4:
            all_errorous_word_preds[i] = errorous_word_preds

    return all_real_word_preds, all_errorous_word_preds



class MinEditDistance:
    def __init__(self, vocab_set: Set[str]) -> None:
        self.vocab_set = vocab_set
        self.word_to_min_edit_distance_word: Dict[str, Tuple[int, str]] = {}

    def save_state(self, path: str) -> None:
        with open(path, 'wb') as f:
            pickle.dump(self.word_to_min_edit_distance_word, f, protocol=pickle.HIGHEST_PROTOCOL)

    def load_state(self, path: str) -> None:
        with open(path, 'rb') as f:
            self.word_to_min_edit_distance_word = pickle.load(f)

    def get_min_edit_distance_words(self, word: str) -> Tuple[int, List[str]]:
        """
        For a given word return all words in vocab_set with the minimum edit distance and the minimum edit distance.
        """
        if word in self.vocab_set:
            return (0, [word])
        
        if word in self.word_to_min_edit_distance_word:
            return self.word_to_min_edit_distance_word[word]
        

        min_edit_distance = float('inf')
        min_edit_distance_words = []
        for vocab_word in self.vocab_set:
            edit_distance = editdistance.eval(word, vocab_word)
            if edit_distance < min_edit_distance:
                min_edit_distance = edit_distance
                min_edit_distance_words = [vocab_word]
            elif edit_distance == min_edit_distance:
                min_edit_distance_words.append(vocab_word)

        self.word_to_min_edit_distance_word[word] = (min_edit_distance, min_edit_distance_words)
        return (min_edit_distance, min_edit_distance_words)


def augment_real_word_with_min_edit_distance(real_word_preds: List[str],
                                             errorous_word_preds: List[str],
                                             vocab_set: Set[str],
                                             med_getter: MinEditDistance) -> List[str]:
    """
    Given real_words and errorous_words for one curve, augment real_words with
    words with minimum edit distance to errorous_words.
    
    Arguments:
    ----------
    real_word_preds: List[str]
        A list of real words sorted by -log_probability in ascending order.
    errorous_word_preds: List[str]
        A list of errorous words sorted by -log_probability in ascending order.
    vocab_set: Set[str]
        A set of all possible words.

    Returns:
    --------
    augmented_real_word_preds: List[str]
        A list of real words sorted by -log_probability in ascending order.
        The list is augmented with words with minimum edit distance to the real words.
    """

    if len(real_word_preds) == 4:
        return real_word_preds
    
    n_with_med_1 = 0
    aug_candidates = []
    
    for errorous_word in errorous_word_preds:
        min_edit_distance, min_edit_distance_words = med_getter.get_min_edit_distance_words(errorous_word)
        for min_edit_distance_word in min_edit_distance_words:
            if min_edit_distance_word in real_word_preds:
                continue
            heapq.heappush(aug_candidates, (min_edit_distance, min_edit_distance_word))
            if min_edit_distance == 1:
                n_with_med_1 += 1
            if n_with_med_1 >= 4 - len(real_word_preds):
                break

    while len(real_word_preds) < 4 and len(aug_candidates) > 0:
        _, word = heapq.heappop(aug_candidates)
        real_word_preds.append(word)


    return real_word_preds



def augment_all_real_words_with_min_edit_distance(all_real_word_preds: List[List[str]],
                                                  all_errorous_word_preds: Dict[int, List[str]],
                                                    vocab_set: Set[str],
                                                    med_getter: MinEditDistance) -> List[List[str]]:
    
    for i in tqdm(all_errorous_word_preds):
        try:
            all_real_word_preds[i] = augment_real_word_with_min_edit_distance(all_real_word_preds[i],
                                                                            all_errorous_word_preds[i],
                                                                            vocab_set,
                                                                            med_getter)
            # med_getter.save_state(MED_GETTER_PATH)
        except KeyboardInterrupt:
            print("Досрочно прервано пользователем")
            break
    return all_real_word_preds



def raw_preds_to_augmented(preds: List[List[Tuple[float, str]]],
                           vocab_set: Set[str],
                           med_getter: MinEditDistance) -> List[List[str]]:
    
    all_real_word_preds, all_errorous_word_preds = get_real_and_errorous_words(preds, vocab_set)
    augmented_preds = augment_all_real_words_with_min_edit_distance(
        all_real_word_preds,
        all_errorous_word_preds,
        vocab_set,
        med_getter)
    return augmented_preds

In [141]:
test_raw_preds_path = os.path.join(DATA_ROOT, f"test_raw_pred___{default_model_fname}__{extra_model_fname}.pkl")

with open(test_raw_preds_path, 'rb') as f:
    raw_test_preds = pickle.load(f)

In [142]:
import copy
raw_test_preds__portion = copy.deepcopy(raw_test_preds)
raw_test_preds__portion = raw_test_preds__portion[:30]

In [143]:
import copy
raw_test_preds__copy = copy.deepcopy(raw_test_preds)

In [144]:
MED_GETTER_PATH = os.path.join(DATA_ROOT, "med_getter.pkl")

med_getter = MinEditDistance(vocab_set)

if os.path.exists(MED_GETTER_PATH):
    med_getter.load_state(MED_GETTER_PATH)

In [148]:
len(med_getter.word_to_min_edit_distance_word)

15888

In [146]:
med_getter.word_to_min_edit_distance_word['сколька']

(1, ['скольки', 'скользка', 'сколка', 'сколько', 'колька', 'скольку'])

In [147]:
augmented_preds = raw_preds_to_augmented(raw_test_preds__copy,
                                         vocab_set,
                                         med_getter)

  0%|          | 0/3449 [00:00<?, ?it/s]

100%|██████████| 3449/3449 [5:19:33<00:00,  5.56s/it]  


In [149]:
med_getter.save_state(MED_GETTER_PATH)

In [150]:
med_getter.word_to_min_edit_distance_word['сколька']

(1, ['скольки', 'скользка', 'сколка', 'сколько', 'колька', 'скольку'])

In [151]:
raw_test_preds__copy[3:]

[[(0.0013753531478262933, 'сколько'),
  (1.9615308022011957, 'скольки'),
  (2.8440072564526417, 'сколька'),
  (3.194628440344293, 'скольком'),
  (3.201470396549101, 'сколькой'),
  (3.5084652859066723, 'сколькая'),
  (3.5100843884467126, 'сколькое'),
  (4.177370122623405, 'скольких')],
 [(0.395288897291407, 'демать'),
  (0.475503677896389, 'дремать'),
  (0.8078159021687566, 'донимать'),
  (0.8789971655982706, 'деписать'),
  (1.2243126288622783, 'донемать'),
  (1.241175843152039, 'делать'),
  (1.423367298033554, 'донсать'),
  (1.595367597788382, 'депать')],
 [(0.004229596896038856, 'не'),
  (2.529828758738902, 'нее'),
  (2.9474920494903927, 'ну'),
  (3.194234329064784, 'на'),
  (4.384369680305218, 'неа'),
  (4.440322895184454, 'нет'),
  (5.9319712284099015, 'неее'),
  (6.310146697510408, 'нае')],
 [(0.00019537568691156427, 'как'),
  (3.8395832056777914, 'кака'),
  (4.326307366375044, 'какие'),
  (4.372576756380138, 'каки'),
  (4.528875542025166, 'каке'),
  (4.6347262115302374, 'како'),
 

In [152]:
augmented_preds

[['на', 'неа', 'не', 'нас'],
 ['что', 'сто', 'чтоб', 'сито'],
 ['опоздания', 'опоздание', 'опоздании', 'опозданий'],
 ['сколько', 'скольки', 'скольких', 'колькой'],
 ['дремать', 'донимать', 'делать', 'девать'],
 ['не', 'нее', 'ну', 'на'],
 ['как', 'кака', 'какие', 'како'],
 ['садовод', 'садовода', 'садовая', 'садовая'],
 ['заметил', 'заметили', 'заметила', 'замерил'],
 ['ваши', 'ваги', 'аги', 'вани'],
 ['ок', 'он', 'оке', 'ока'],
 ['плинтус', 'поленница', 'плинтус', 'плинтус'],
 ['ай', 'ау', 'вай', 'айк'],
 ['ищем', 'ищет', 'идем', 'ире'],
 ['они', 'лет', 'он', 'оно'],
 ['могу', 'могут', 'могул', 'мог'],
 ['может', 'можете', 'молет', 'можно'],
 ['спокойный', 'спокойной', 'спокойные', 'спокойней'],
 ['рядом', 'ряды', 'рядов', 'ржав'],
 ['вспомнить', 'вспомнилась', 'вспомнил', 'вспоминать'],
 ['максим', 'макам', 'макаем', 'маками'],
 ['веселое', 'веселом', 'веселов', 'веселина'],
 ['неведомым', 'неведомыми', 'невесомым', 'невесомыми'],
 ['туда', 'труда', 'тула', 'мудра'],
 ['тебя', 'тебе

In [153]:
augmented_preds__copy = copy.deepcopy(augmented_preds)

augment_list = None
with open(r"..\data\submissions\sample_submission.csv", 'r', encoding = 'utf-8') as f:
    augment_lines = f.read().splitlines()
augment_list = [line.split(",") for line in augment_lines]

augment_submission(augmented_preds__copy, augment_list)

create_submission(augmented_preds__copy, "sub_full_min_edit_dis")

In [115]:
real, err = get_real_and_errorous_words(raw_test_preds, vocab_set)

In [116]:
augment_submission(real, augment_list)

In [117]:
create_submission(real, "sub_without_min_edit")

# Обработка дефиса

In [81]:
default_model_fname = "best_model__2023_11_04__18_31_37__0.02530_default_switch_2.pt"
extra_model_fname = "best_model__2023_11_05__07_55_13__0.02516_extra_switch_2__with_pad_cutting.pt"

test_raw_preds_path = os.path.join(DATA_ROOT, f"test_raw_pred___{default_model_fname}__{extra_model_fname}.pkl")

with open(test_raw_preds_path, 'rb') as f:
    raw_test_preds = pickle.load(f)

In [82]:
def get_vocab_set(vocab_path: str):
    with open(vocab_path, 'r', encoding = "utf-8") as f:
        return set(f.read().splitlines())

In [83]:
vocab_set = get_vocab_set(os.path.join(DATA_ROOT, "voc.txt"))

In [84]:
defis_set = {el for el in vocab_set if '-' in el}

In [85]:
real, err = get_real_and_errorous_words(raw_test_preds, vocab_set)

In [86]:
import editdistance

from tqdm import tqdm

In [87]:

for i in tqdm(range(len(real))):
    if not real[i]:
        for err_el in err[i]:
            for defis_word in defis_set:
                if editdistance.distance(err_el, defis_word) == 1:
                    real[i].append(defis_word)
                    print(i)
                    break
            

  0%|          | 0/10000 [00:00<?, ?it/s]

  6%|▋         | 626/10000 [00:05<02:03, 76.12it/s] 

625


 17%|█▋        | 1678/10000 [00:09<00:47, 176.90it/s]


KeyboardInterrupt: 

In [78]:
augment_list = None
with open(r"..\data\submissions\sample_submission.csv", 'r', encoding = 'utf-8') as f:
    augment_lines = f.read().splitlines()
augment_list = [line.split(",") for line in augment_lines]

In [79]:
augment_submission(real, augment_list)

In [80]:
create_submission(real, "submit_with_defis")

# Table for predict.py

In [1]:
import pandas as pd

In [50]:
df = pd.DataFrame(columns = [
            'predictor_id', 'prediction', 'model_name', 'model_weights', 'generator_name', 'grid_name', 'dataset_split'])
df = df.set_index('predictor_id')
df

,prediction,model_name,model_weights,generator_name,grid_name,dataset_split
predictor_id,,,,,,


In [67]:
generator_kwargs = {
            "max_steps_n": 35,
            "return_hypotheses_n": 7,
            "beamsize": 6,
            "normalization_factor": 0.5,
            "verbose": False
        }

In [68]:
from dataclasses import dataclass, asdict
from typing import Tuple, List

@dataclass
class Prediction:
    prediction: Tuple[int, List[Tuple[float, str]]]
    model_name: str
    model_weights: str
    generator_name: str
    generator_kwargs: dict
    grid_name: str
    dataset_split: str

In [77]:
def prediction_to_dict(prediction: Prediction):
    d = asdict(prediction)
    del d['prediction']
    del d['generator_kwargs']
    for k, v in prediction.generator_kwargs.items():
        d[f'{prediction.generator_name}__{k}'] = v
    return d

In [78]:
prediction_lsit = [Prediction(f'pred{i}', f'model{i}', f'model_w{i}', f'gen{i}', generator_kwargs, 'default', 'test')
                   for i in range(10)]

In [79]:
prediction_dict_lsit = [prediction_to_dict(pred) for pred in prediction_lsit]
prediction_dict_lsit

[{'model_name': 'model0',
  'model_weights': 'model_w0',
  'generator_name': 'gen0',
  'grid_name': 'default',
  'dataset_split': 'test',
  'gen0__max_steps_n': 35,
  'gen0__return_hypotheses_n': 7,
  'gen0__beamsize': 6,
  'gen0__normalization_factor': 0.5,
  'gen0__verbose': False},
 {'model_name': 'model1',
  'model_weights': 'model_w1',
  'generator_name': 'gen1',
  'grid_name': 'default',
  'dataset_split': 'test',
  'gen1__max_steps_n': 35,
  'gen1__return_hypotheses_n': 7,
  'gen1__beamsize': 6,
  'gen1__normalization_factor': 0.5,
  'gen1__verbose': False},
 {'model_name': 'model2',
  'model_weights': 'model_w2',
  'generator_name': 'gen2',
  'grid_name': 'default',
  'dataset_split': 'test',
  'gen2__max_steps_n': 35,
  'gen2__return_hypotheses_n': 7,
  'gen2__beamsize': 6,
  'gen2__normalization_factor': 0.5,
  'gen2__verbose': False},
 {'model_name': 'model3',
  'model_weights': 'model_w3',
  'generator_name': 'gen3',
  'grid_name': 'default',
  'dataset_split': 'test',
  'g

In [82]:
df2 = pd.DataFrame(prediction_dict_lsit)
df2.index.names = ['perdictor_id']
df2

,model_name,model_weights,generator_name,grid_name,dataset_split,gen0__max_steps_n,gen0__return_hypotheses_n,gen0__beamsize,gen0__normalization_factor,gen0__verbose,...,gen8__max_steps_n,gen8__return_hypotheses_n,gen8__beamsize,gen8__normalization_factor,gen8__verbose,gen9__max_steps_n,gen9__return_hypotheses_n,gen9__beamsize,gen9__normalization_factor,gen9__verbose
perdictor_id,,,,,,,,,,,,,,,,,,,,,
0,model0,model_w0,gen0,default,test,35.0,7.0,6.0,0.5,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,model1,model_w1,gen1,default,test,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,model2,model_w2,gen2,default,test,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,model3,model_w3,gen3,default,test,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,model4,model_w4,gen4,default,test,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,model5,model_w5,gen5,default,test,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,model6,model_w6,gen6,default,test,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,model7,model_w7,gen7,default,test,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,model8,model_w8,gen8,default,test,NaN,NaN,NaN,NaN,NaN,...,35.0,7.0,6.0,0.5,False,NaN,NaN,NaN,NaN,NaN


In [45]:
df2.loc[df2['prediction'] == 'pred0', 'dataset_split'] = 'train'

In [46]:
df2

,prediction,model_name,model_weights,generator_name,grid_name,dataset_split
predictor_id,,,,,,
0,pred0,model0,model_w0,gen0,default,train
1,pred1,model1,model_w1,gen1,default,test
2,pred2,model2,model_w2,gen2,default,test
3,pred3,model3,model_w3,gen3,default,test
4,pred4,model4,model_w4,gen4,default,test
5,pred5,model5,model_w5,gen5,default,test
6,pred6,model6,model_w6,gen6,default,test
7,pred7,model7,model_w7,gen7,default,test
8,pred8,model8,model_w8,gen8,default,test


In [54]:
n_lines_of_same_predictor = len(df2.loc[df2['prediction'] == 'pred0'])

assert len(df2.loc[df2['prediction'] == 'pred0']) <= 1, "super trouble"

if 

In [59]:
prediction = Prediction(i, f'pred{i}', f'model{i}', f'model_w{i}', f'gen{i}', 'default', 'test')

In [64]:
asdict(prediction)

{'predictor_id': 0,
 'prediction': 'pred0',
 'model_name': 'model0',
 'model_weights': 'model_w0',
 'generator_name': 'gen0',
 'grid_name': 'default',
 'dataset_split': 'test'}

In [63]:
prediction_to_dict(prediction)

KeyError: 'generator_kwargs'

# ONNX export

In [2]:
import torch

from model import get_m1_bigger_model


weights_path = '../data/trained_models/m1_bigger/m1_bigger_v2__2023_11_12__12_30_29__0.13121__greed_acc_0.86098__default_l2_0_ls0_switch_2.pt'
model = get_m1_bigger_model('cpu', weights_path)

scripted_model = torch.jit.script(model)


In [17]:
seq_len = 32
batch_size = 10
n_coord_feats=6
char_emb_dim=128
out_seq_len = 10
kb_token_dim = 66

traj_feat_seq = torch.rand([seq_len, batch_size, n_coord_feats])
kb_tokens = torch.ones([seq_len, batch_size], dtype = torch.int)
out_char_torkens = torch.ones(*[out_seq_len, batch_size], dtype = torch.int)
x_pad_mask = torch.zeros([batch_size, seq_len],  dtype=torch.bool)
y_pad_mask = torch.zeros([batch_size, out_seq_len],  dtype=torch.bool)
# Dummy input data for export (needed for capturing types and shapes)
example_input = (traj_feat_seq, kb_tokens, out_char_torkens, x_pad_mask, y_pad_mask)


In [22]:
onnx_path = "model.onnx"
torch.onnx.export(model, example_input, onnx_path)

In [23]:
onnx_path = "scripted_model.onnx"
torch.onnx.export(scripted_model, example_input, onnx_path)

UnsupportedOperatorError: Exporting the operator 'aten::format' to ONNX opset version 17 is not supported. Please feel free to request support or submit a pull request on PyTorch GitHub: https://github.com/pytorch/pytorch/issues.